In [1]:
from google.colab import files
uploaded = files.upload()

Saving all_reviews.txt to all_reviews.txt


In [2]:
import nltk

In [3]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [4]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [5]:
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


True

In [7]:
import pandas as pd
import numpy as np
import string
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score, classification_report

In [10]:
df = pd.read_csv('all_reviews.txt', sep='\t', header=None)

In [11]:
df.head()

,0,1
0,So there is no way for me to plug it in here i...,0
1,Good case Excellent value,1
2,Great for the jawbone,1
3,Tied to charger for conversations lasting more...,0
4,The mic is great,1


In [12]:
df.columns = ['Review', 'Sentiment']

In [13]:
df.head()

,Review,Sentiment
0,So there is no way for me to plug it in here i...,0
1,Good case Excellent value,1
2,Great for the jawbone,1
3,Tied to charger for conversations lasting more...,0
4,The mic is great,1


In [14]:
df.shape

(3000, 2)

In [31]:
def text_processing(documents):
  n = len(documents)
  # removing punctuations
  print("Removing Punctuations")
  table = str.maketrans('','',string.punctuation)
  for i in range(n):
    documents[i] = documents[i].translate(table)
  print("Done Removing Punctuation")
  print(documents[0])
  
  # tokenization
  print("Performing Tokenization...")
  tokens = []
  for i in range(n):
    tokens.append(word_tokenize(documents[i].lower()))
  print("Done Tokenization")
  print(tokens[0])
  
  # removing stopwords
  print("Removing Stopwords")
  english_stopwords = stopwords.words("english")
  wordsList = []
  for i in range(n):
    words = []
    for token in tokens[i]:
      if token not in english_stopwords:
        words.append(token)
    wordsList.append(words)
  print("Stopwords Removed")
  print(wordsList[0])

  print("Lemmatization...")
  wnet = WordNetLemmatizer()
  for i in range(n):
    for j in range(len(wordsList[i])):
      wordsList[i][j] = wnet.lemmatize(wordsList[i][j], pos='v')
  print("Done Lemmatization...")
  
  for i in range(n):
    wordsList[i] = " ".join(wordsList[i])
    
  return wordsList

In [32]:
wordsList = text_processing(df['Review'])

Removing Punctuations


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys


Done Removing Punctuation
So there is no way for me to plug it in here in the US unless I go by a converter
Performing Tokenization...
Done Tokenization
['so', 'there', 'is', 'no', 'way', 'for', 'me', 'to', 'plug', 'it', 'in', 'here', 'in', 'the', 'us', 'unless', 'i', 'go', 'by', 'a', 'converter']
Removing Stopwords
Stopwords Removed
['way', 'plug', 'us', 'unless', 'go', 'converter']
Lemmatization...
Done Lemmatization...


In [33]:
wordsList[0]

'way plug us unless go converter'

In [38]:
tfidf = TfidfVectorizer()
vectors = tfidf.fit_transform(wordsList).toarray()

In [39]:
x_train, x_test, y_train, y_test = train_test_split(vectors, df['Sentiment'], test_size=0.25)

In [40]:
x_train.shape

(2250, 4532)

In [41]:
logistic = LogisticRegression()
logistic.fit(x_train, y_train)

LogisticRegression()

In [42]:
y_pred = logistic.predict(x_test)

In [43]:
accuracy_score(y_test, y_pred)

0.788

In [44]:
from sklearn.metrics import confusion_matrix

In [45]:
confusion_matrix(y_test, y_pred)

array([[309,  63],
       [ 96, 282]])